import merged shots

In [10]:
import pandas as pd
import networkx as nx
from pyvis.network import Network

In [2]:
def add_player_node(G, team, player_id):
    """
    Add a player node with the real player ID if it doesn't exist.
    Node label example: "CHI_8474141".
    """
    node_key = f"{team}_{player_id}"
    if not G.has_node(node_key):
        G.add_node(node_key, 
                   team=team, 
                   player_id=player_id, 
                   type='player')
    return node_key

def add_shot_node(G, shot_id, is_goal, game_id):
    """
    Add a shot node with an is_goal flag and game_id attribute.
    """
    G.add_node(shot_id, 
               type='shot', 
               is_goal=is_goal, 
               game_id=game_id)
    return shot_id

def create_hockey_network():
    """Create an empty directed graph for hockey analysis."""
    return nx.DiGraph()

In [7]:
def process_game_data(df):
    """Process game data from a Pandas DataFrame and create a network graph."""
    # Create directed graph
    G = create_hockey_network()
    
    # Dictionary to track how often one player passes to another
    passing_counts = {}
    
    # Process each shot attempt in the DataFrame
    for idx, row in df.iterrows():
        # Build a unique shot ID for the node (e.g., shot_0, shot_1, etc.)
        shot_id = f"shot_{idx}"
        
        # Flag if this shot was a goal
        is_goal = (row['G?'] == 'y')
        
        # Capture the game ID from your dataset
        game_id = row['Game ID']
        
        # Create the shot node in the graph
        add_shot_node(G, shot_id, is_goal, game_id)
        
        # Add shooter node using the real player ID
        shooter_id = add_player_node(G, row['Team'], row['Shooter'])
        
        # Connect shooter to this shot node
        G.add_edge(shooter_id, shot_id, relationship_type='MAKES_SHOT')
        
        # Process the assists in reverse order (A3 -> A2 -> A1), ignoring NaNs
        passers = []
        for assist_col in ['A3', 'A2', 'A1']:
            if pd.notna(row[assist_col]):
                passer_id = add_player_node(G, row['Team'], row[assist_col])
                passers.append(passer_id)
        
        # If there were any assists, append the shooter to form the passing chain
        if passers:
            passers.append(shooter_id)
            
            # Track consecutive passes (A3->A2->A1->Shooter)
            for i in range(len(passers) - 1):
                passer = passers[i]
                receiver = passers[i + 1]
                pass_key = (passer, receiver)
                passing_counts[pass_key] = passing_counts.get(pass_key, 0) + 1
    
    # After processing all shots, add weighted passing relationships
    for (passer, receiver), weight in passing_counts.items():
        G.add_edge(passer, receiver, relationship_type='PASSES_TO', weight=weight)
    
    return G

In [6]:
shots.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206 entries, 0 to 205
Data columns (total 29 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Period            206 non-null    int64  
 1   Time              206 non-null    object 
 2   Strength          206 non-null    object 
 3   Team              206 non-null    object 
 4   Shooter           206 non-null    float64
 5   Shot Type?        206 non-null    object 
 6   A1                173 non-null    float64
 7   A2                92 non-null     float64
 8   A3                37 non-null     float64
 9   A1 Zone           173 non-null    object 
 10  A2 Zone           92 non-null     object 
 11  A3 Zone           37 non-null     object 
 12  SC?               81 non-null     object 
 13  SOG?              134 non-null    object 
 14  Screen            11 non-null     object 
 15  Situation         148 non-null    object 
 16  Origin            144 non-null    object 
 1

In [23]:
shots = pd.read_csv('merged_shots.csv')

shots = shots.loc[shots['Game ID'] == 20116]

G = process_game_data(shots)

In [24]:

def visualize_passing_network(G, output_html="passing_network.html"):
    """
    Generate an interactive PyVis visualization of the passing network.
    Includes only player nodes and edges with relationship_type='PASSES_TO'.
    
    :param G: A NetworkX DiGraph containing nodes (players, shots) and edges (PASSES_TO, MAKES_SHOT).
    :param output_html: The file path/name for the output HTML visualization.
    """
    # Initialize PyVis network; you can adjust parameters like height, width, etc.
    net = Network(height="750px", width="100%", bgcolor="#222222", font_color="white", notebook=False)
    
    # Filter: Only pass edges of type 'PASSES_TO'
    passing_edges = [
        (u, v, data)
        for u, v, data in G.edges(data=True)
        if data.get("relationship_type") == "PASSES_TO"
    ]
    
    # Gather the player nodes that appear in these edges
    player_nodes = set()
    for u, v, data in passing_edges:
        player_nodes.add(u)
        player_nodes.add(v)
    
    # Add only the player nodes to the PyVis network
    for node in player_nodes:
        node_data = G.nodes[node]
        if node_data.get("type") == "player":
            # You can customize label, color, or any PyVis parameters here
            net.add_node(
                node,
                label=node,
                title=f"Team: {node_data.get('team')}<br>ID: {node_data.get('player_id')}",
                color="#FFD700"  # Example: Golden color for players
            )
    
    # Add edges (PASSES_TO) with a 'weight' that can determine edge thickness in PyVis
    for u, v, data in passing_edges:
        if (u in player_nodes) and (v in player_nodes):
            weight = data.get("weight", 1)
            net.add_edge(u, v, value=weight, title=f"Pass count: {weight}")
    
    # Generate the interactive HTML file
    net.save_graph(output_html)
    print(f"Interactive passing network saved to {output_html}")

In [37]:
from pyvis.network import Network

def visualize_passing_network(G, team_code=None, output_html="passing_network.html"):
    """
    Generate an interactive PyVis visualization of the passing network,
    optionally filtering by a specified team_code.
    
    :param G:         A NetworkX DiGraph (nodes include type='player' or type='shot').
    :param team_code: Optional string indicating which team's subgraph to visualize (e.g., "CHI").
    :param output_html: Filename for output HTML visualization.
    """

    # Create a directed PyVis network
    net = Network(
        height="750px",
        width="100%",
        bgcolor="#222222",
        font_color="white",
        directed=True,
        notebook=False
    )
    
    
    #net.show_buttons(filter_=["physics"])
    net.show_buttons()

    # 1. Identify edges of interest: 'PASSES_TO'
    #    If team_code is given, we only include edges where both player nodes share the same team_code.
    passing_edges = []
    for u, v, data in G.edges(data=True):
        if data.get("relationship_type") == "PASSES_TO":
            # Only include edges if both nodes are players and optionally match the team_code
            if G.nodes[u].get("type") == "player" and G.nodes[v].get("type") == "player":
                if team_code:
                    # Ensure both nodes match the specified team_code
                    if G.nodes[u].get("team") == team_code and G.nodes[v].get("team") == team_code:
                        passing_edges.append((u, v, data))
                else:
                    # If no team filter, just add all 'PASSES_TO' edges
                    passing_edges.append((u, v, data))

    # 2. Gather relevant player nodes
    player_nodes = set()
    for u, v, data in passing_edges:
        player_nodes.add(u)
        player_nodes.add(v)
        
    # 3. Add the filtered player nodes to the PyVis graph
    #    We'll label them with team and ID so the user sees something descriptive.
    for node in player_nodes:
        node_data = G.nodes[node]
        net.add_node(
            node,
            label=node,  # or something else, e.g. node_data.get('player_id')
            title=f"Team: {node_data.get('team')}<br>ID: {node_data.get('player_id')}",
            color="#FFD700" if team_code is None else "#00FFAA",  # Example color schemes
        )

    # 4. Add edges (with their weight) as labels, so pass counts are visible without hover
    #    You can also adjust edge colors, arrow styles, etc.
    for u, v, data in passing_edges:
        weight = data.get("weight", 1)
        net.add_edge(
            source=u,
            to=v,
            #label=str(weight),   # Show the pass count directly on the edge
            value=weight,       # This can affect thickness if "physics" is enabled
            color="#BBE1FA",    # Example color for edges
            
            arrowStrikethrough=False
        )

    # 5. Generate the interactive HTML file
    net.save_graph(output_html)
    print(f"Interactive passing network saved to {output_html}")


In [38]:
# Visualize the passing network:
visualize_passing_network(G, team_code='EDM', output_html="edm_passing_network.html")

Interactive passing network saved to edm_passing_network.html
